In [1]:
# Import the tables of the data set as dataframes.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns

BASE_DATA_DIR = "../../data"
ORIGINAL_DATA_DIR = f"{BASE_DATA_DIR}/original/data"
KAGGLE_PATH = "/kaggle/input/filess"

users = pd.read_csv("{}/users.csv.gz".format(ORIGINAL_DATA_DIR), dtype={"gender": "category"})
events = pd.read_csv(
    "{}/events.csv.gz".format(ORIGINAL_DATA_DIR),
    dtype={"event_type": "category"},
    parse_dates=["event_date"]
)
transactions = pd.read_csv(
    "{}/transactions.csv.gz".format(ORIGINAL_DATA_DIR),
    parse_dates=["start_time", "commit_time"]
)
documents = pd.read_csv(
    "{}/documents.csv.gz".format(ORIGINAL_DATA_DIR)
)

Feature engineering is an important step in our project. As a matter of fact, the datasets containing the information from the Learnavi users needed to be manipulated so to extract those features that could indicate the level of knowledge of the student and allow to classify what the user is going to answer.

The features we have extracted are the following:
1) the mastery level of the user for each topic;
2) the mean mastery level of the user;
3) the sum of the scores of the student on the questions answered, normalized by the number of questions answered;
4) the sum of the scores on the questions weighted on the difficulty, normalized by the number of questions answered.


It must be noted that the difficulty of each question and the associated scores were also extracted from the Learnavi datasets, as the reader will found explained below.

Limitations of the extracted features are discussed in the report and at the end of this notebook.

### Mastery level for each topic

In order to extract the mastery level for each topic, we have followed the following steps:
1) the events "NAVIGATE_DASHBOARD" were extracted from the dataset;
2) among these events, the ones that contained the information about the mastery level (present in the dashboard) were selected;
3) among these, the ones pertaining to the German topic were selected;
4) the most recent mastery level scores for each user were selected.

In [2]:
#isolating only the events "NAVIGATE_DASHBOARD"
data_mastery = events[events["action"] == "NAVIGATE_DASHBOARD"]
data_mastery.shape

(1093791, 13)

In [3]:
# select only the the ones that hold dashboard in the tracking_data
import json
def has_dash(x):
    j = json.loads(x)
    return "dashboard" in j

d2 = data_mastery[data_mastery["tracking_data"].apply(has_dash)].copy()

In [4]:
#introduce a variable that states the topic
def get_subject(doc):
    res = None
    doc = json.loads(doc)
    res = doc.get('dashboard').get('title')
    return res
d2['subject'] = d2['tracking_data'].apply(lambda d2: get_subject(d2))

#select only rows with mastery scores in the german topic
d3 = d2[d2["subject"]=="Deutsch"]
d3.to_csv(f"{BASE_DATA_DIR}/preprocessed/events_german.csv")

In [6]:
## get most recent one for each user_id
d5 = d3.sort_values(["user_id", "event_date"],ascending=False).groupby("user_id").apply(lambda x: x.iloc[0])

# get the mastery scores for each user
def load_mastery(x):
    j = json.loads(x['tracking_data'])
    if "dashboard" in j:
        return pd.Series(
            [
                child["userData"]["mastery"]
                for topic in j["dashboard"]["topics"]
                for child in topic["children"]
            ],
            name=x["user_id"], dtype=float
        )
    return pd.Series([], dtype=float)
dm6 = d5.apply(load_mastery,axis=1)

#rename columns
j = json.loads(d5['tracking_data'].iloc[0])
ls2 = [child["topic"]["name"] for topic in j["dashboard"]["topics"] for child in topic["children"]]
dm6 = dm6.set_axis(ls2, axis=1)

In [7]:
## get most recent one for each user_id
d5 = d3.sort_values(["user_id", "event_date"],ascending=False).groupby("user_id").apply(lambda x: x.iloc[0])

# get the mastery scores for each user
def load_mastery(x):
    j = json.loads(x['tracking_data'])
    if "dashboard" in j:
        return pd.Series(
            [
                child["userData"]["mastery"]
                for topic in j["dashboard"]["topics"]
                for child in topic["children"]
            ],
            name=x["user_id"], dtype=float
        )
    return pd.Series([], dtype=float)
dm6 = d5.apply(load_mastery,axis=1)

#rename columns
j = json.loads(d5['tracking_data'].iloc[0])
ls2 = [child["topic"]["name"] for topic in j["dashboard"]["topics"] for child in topic["children"]]
dm6 = dm6.set_axis(ls2, axis=1)

#save to csv
dm6.to_csv(f"{BASE_DATA_DIR}/lernnavi/mastery_per_topic.csv")

In [8]:
d5_with_dates = d3.sort_values(["user_id", "event_date"],ascending=False).set_index(["user_id", "event_date"], drop=False)

dm6_with_dates = d5_with_dates.apply(load_mastery, axis=1)

# rename columns
j = json.loads(d5_with_dates['tracking_data'].iloc[0])
ls2 = [child["topic"]["name"] for topic in j["dashboard"]["topics"] for child in topic["children"]]
dm6_with_dates = dm6_with_dates.set_axis(ls2, axis=1)

dm6_with_dates.to_csv(f"{BASE_DATA_DIR}/lernnavi/mastery_per_topic_with_dates.csv")

As said before, the mastery level scores are the most recent ones. We are aware that selecting the most recent scores is a limitation. As a matter of fact, in the next milestone at which point in the student timeline consider the mastery level scores will be defined properly. 

### Mean mastery score of the student

This feature is the mean of the mastery level scores of the user for each topic. This feature was selected as it gives an overall idea of the knowledge of the users and their ability to learn and improve.

In [9]:
mean_mastery = dm6.mean(axis=1)
mean_mastery.to_csv(f"{BASE_DATA_DIR}/lernnavi/mean_mastery.csv")

In [10]:
mean_mastery.head(10)

user_id
387604    0.177370
387605    0.098331
387615    0.096108
387643    0.048141
387644    0.111754
387650    0.000000
387655    0.044300
387666    0.050403
387667    0.006944
387668    0.012153
dtype: float64

### Scores of the students 


Two scores were extracted from the dataset and assigned to each student: 
1) the sum of the scores of the student on the questions answered, normalized by the number of questions answered;
2) the sum of the scores of the student on the questions answered weighted on the difficulty, normalized by the number of questions answered.

The score for each question were as well extracted from the dataset. 1 was assigned to each question evaluated as correct, 0.5 as partially correct, 0 as incorrect. The score of the student on the questions was then calculated as the sum of the scores of the questions answered, normalized by the number of questions answered.

In [11]:
## assigning a score for each evaluation
def assign_score(evaluation):
    if evaluation=="CORRECT":
        score=1
    elif evaluation=="PARTIAL":
        score=0.5
    else:
        score=0
    return score

In [12]:
## merging the transactions with the documents, retaining only the rows that could be useful, 
#as determined in the exploratory
types_we_can_use = [
    "CLOZE_MATH",
    "CLOZE_TEXT",
    "CLOZE_TEXT_DROPDOWN",
    "DND_IN_TEXT",
    "FIX_TEXT",
    "HIGHLIGHT",
    "MATH_STEP_BY_STEP",
    "MULTIPLE_CHOICE",
    "OPEN_TASK",
    "SOLUTION_FIELD",
]
merged = (
    transactions.merge(
        documents[[
            "version",
            "document_id",
            "type",
            "content",
            "topic_id"
        ]].rename({"version": "document_version"}, axis=1),
        on=["document_id", "document_version"],
    )
)

merged = merged[merged["type_x"].isin(types_we_can_use)]
merged.head(2)

,transaction_id,transaction_token,user_id,document_id,document_version,evaluation,input,start_time,commit_time,user_agent,...,session_closed,session_type,session_accepted,challenge,challenge_id,challenge_order,challenge_name,type_y,content,topic_id_y
29,688415,61eb829d-bdda-4107-86af-ad9a14a7bdc9,393211,9wk5dtV2mF59odW0wCEYYc,75003,PARTIAL,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""Person...",2021-05-21 08:07:37.048,2021-05-21 08:13:30.953999872,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,...,NaN,NaN,NaN,True,1083.0,4.0,G3h – Training Rhetorik,CLOZE_TEXT_INPUT,"{""id"": ""9wk5dtV2mF59odW0wCEYYc"", ""type"": ""CLOZ...",4256
31,695249,fe8b691e-4d24-4583-b9c1-0767f27bb4ec,391665,9wk5dtV2mF59odW0wCEYYc,75003,PARTIAL,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""Person...",2021-05-26 16:34:02.276,2021-05-26 16:47:27.720000000,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3...,...,NaN,NaN,NaN,True,1083.0,4.0,G3h – Training Rhetorik,CLOZE_TEXT_INPUT,"{""id"": ""9wk5dtV2mF59odW0wCEYYc"", ""type"": ""CLOZ...",4256
33,695533,e62dfb0d-1b21-420f-817d-b579fb307e93,393288,9wk5dtV2mF59odW0wCEYYc,75003,WRONG,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""Metaph...",2021-05-26 19:57:16.109,2021-05-26 20:08:16.446000128,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,...,NaN,NaN,NaN,True,1083.0,4.0,G3h – Training Rhetorik,CLOZE_TEXT_INPUT,"{""id"": ""9wk5dtV2mF59odW0wCEYYc"", ""type"": ""CLOZ...",4256
34,696498,f68ed455-4fd6-4662-814f-50bebb11f9d0,393304,9wk5dtV2mF59odW0wCEYYc,75003,PARTIAL,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""Person...",2021-05-27 21:35:34.527,2021-05-27 21:40:46.694000128,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6...,...,NaN,NaN,NaN,True,1083.0,4.0,G3h – Training Rhetorik,CLOZE_TEXT_INPUT,"{""id"": ""9wk5dtV2mF59odW0wCEYYc"", ""type"": ""CLOZ...",4256
35,696507,540a3157-3ae9-40a9-a69b-ccb3cb64ae50,393294,9wk5dtV2mF59odW0wCEYYc,75003,WRONG,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""Pleona...",2021-05-27 22:04:16.945,2021-05-27 22:07:05.331000064,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4...,...,NaN,NaN,NaN,True,1083.0,4.0,G3h – Training Rhetorik,CLOZE_TEXT_INPUT,"{""id"": ""9wk5dtV2mF59odW0wCEYYc"", ""type"": ""CLOZ...",4256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2124804,3934645,91424ad3-2072-4ae9-ad3c-4b21f5af5a3b,431541,9Vh0FkB31BD84G8lStObVY,178219,PARTIAL,"{""type"": ""CLOZE_TEXT_DROPDOWN"", ""clozeInputs"":...",2023-02-05 17:41:52.141,2023-02-05 17:42:44.092999936,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,...,1.0,2.0,NaN,False,NaN,NaN,NaN,CLOZE_TEXT_DROPDOWN,"{""id"": ""9Vh0FkB31BD84G8lStObVY"", ""mode"": ""DEFA...",3142
2124805,3935709,00d6dc78-1534-4837-8619-e804fc7e4821,411250,9Vh0FkB31BD84G8lStObVY,178219,PARTIAL,"{""type"": ""CLOZE_TEXT_DROPDOWN"", ""clozeInputs"":...",2023-02-05 20:53:28.617,2023-02-05 20:54:37.663000064,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,...,1.0,2.0,NaN,False,NaN,NaN,NaN,CLOZE_TEXT_DROPDOWN,"{""id"": ""9Vh0FkB31BD84G8lStObVY"", ""mode"": ""DEFA...",3142
2124808,3934864,5b6ff414-9468-45c1-aff2-fc3e07916cd5,388261,3mrTmlw7MiH7CRW38jd2VX,178139,CORRECT,"{""type"": ""CLOZE_MATH"", ""clozeInputs"": [""-4"", ""...",2023-02-05 18:07:42.229,2023-02-05 18:08:16.424000000,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,...,1.0,2.0,1.0,False,NaN,NaN,NaN,CLOZE_MATH,"{""id"": ""3mrTmlw7MiH7CRW38jd2VX"", ""type"": ""CLOZ...",999
2124813,3935685,5068aaca-3f49-406d-a86f-9c9712b40c65,411250,9TO7mS3JvKL8afeTa3LPZf,178271,PARTIAL,"{""type"": ""CLOZE_TEXT_DROPDOWN"", ""clozeInputs"":...",2023-02-05 20:46:41.710,2023-02-05 20:47:50.582000128,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,...,1.0,2.0,NaN,False,NaN,NaN,NaN,CLOZE_TEXT_DROPDOWN,"{""id"": ""9TO7mS3JvKL8afeTa3LPZf"", ""mode"": ""DEFA...",3142


In [13]:
#merging on the topics 
topics = pd.read_csv(
    f'{ORIGINAL_DATA_DIR}/topics_translated.csv'
)

merged_with_topics = merged.merge(topics, left_on="topic_id_x", right_on="id", how="inner")
merged_with_topics.head(2)

,transaction_id,transaction_token,user_id,document_id,document_version,evaluation,input,start_time,commit_time,user_agent,...,challenge_name,type_y,content,topic_id_y,id,german_name,german_description,name,description,math
0,702616,d7f72afa-066c-49d3-877d-80e4bce9364f,391625,9wk5dtV2mF59odW0wCEYYc,75003,PARTIAL,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""Assona...",2021-06-02 15:48:45.228,2021-06-02 15:52:20.670000128,unknown,...,NaN,CLOZE_TEXT_INPUT,"{""id"": ""9wk5dtV2mF59odW0wCEYYc"", ""type"": ""CLOZ...",4256,2045,Rhetorik,NaN,Rhetoric,NaN,0
1,703810,b20883dc-dc03-4c25-8f0a-b625af28c041,388290,9wk5dtV2mF59odW0wCEYYc,75003,CORRECT,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""Metaph...",2021-06-04 05:50:50.265,2021-06-04 05:51:32.985999872,unknown,...,NaN,CLOZE_TEXT_INPUT,"{""id"": ""9wk5dtV2mF59odW0wCEYYc"", ""type"": ""CLOZ...",4256,2045,Rhetorik,NaN,Rhetoric,NaN,0
2,716303,490023d6-ceeb-4f0d-aa1f-2b207efac1fa,392845,9wk5dtV2mF59odW0wCEYYc,75003,WRONG,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""blumen...",2021-06-15 08:16:58.894,2021-06-15 08:17:43.192000000,unknown,...,NaN,CLOZE_TEXT_INPUT,"{""id"": ""9wk5dtV2mF59odW0wCEYYc"", ""type"": ""CLOZ...",4256,2045,Rhetorik,NaN,Rhetoric,NaN,0
3,727495,023b2fac-9e2b-4c54-9072-09065dddc56d,391776,9wk5dtV2mF59odW0wCEYYc,75003,WRONG,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": ["""", """",...",2021-07-01 08:28:54.139,2021-07-01 08:30:17.124000000,unknown,...,NaN,CLOZE_TEXT_INPUT,"{""id"": ""9wk5dtV2mF59odW0wCEYYc"", ""type"": ""CLOZ...",4256,2045,Rhetorik,NaN,Rhetoric,NaN,0
4,741050,20d9f392-c5ba-4756-bb24-0aad3c78a1b0,393730,6buw4x5E4Yt92KVfYANu0e,75016,CORRECT,"{""type"": ""OPEN_TASK"", ""openInput"": ""Trikolon\n""}",2021-07-30 14:42:42.929,2021-07-30 14:43:25.931000064,unknown,...,NaN,OPEN,"{""id"": ""6buw4x5E4Yt92KVfYANu0e"", ""type"": ""OPEN...",4256,2045,Rhetorik,NaN,Rhetoric,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866706,3473424,a643321c-1117-4275-b6fc-efac3ab3d040,413371,3FzjtkOlOh769ACiGbVzlf,166717,PARTIAL,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""das"", ...",2022-12-14 09:15:06.599,2022-12-14 09:15:50.023000064,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,...,NaN,CLOZE_TEXT_INPUT,"{""id"": ""3FzjtkOlOh769ACiGbVzlf"", ""type"": ""CLOZ...",3283,3249,Inhalt von Nebensätzen,NaN,Content of subordinate clauses,NaN,0
866707,3606166,1d4c1eb3-25fc-473d-9878-225ac31a0390,427529,3FzjtkOlOh769ACiGbVzlf,166717,PARTIAL,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""was"", ...",2023-01-05 22:11:49.222,2023-01-05 22:12:50.103000064,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,...,NaN,CLOZE_TEXT_INPUT,"{""id"": ""3FzjtkOlOh769ACiGbVzlf"", ""type"": ""CLOZ...",3283,3249,Inhalt von Nebensätzen,NaN,Content of subordinate clauses,NaN,0
866708,3812408,a56e13ed-e447-45f0-b80f-5f4044c6e238,404555,3FzjtkOlOh769ACiGbVzlf,166717,PARTIAL,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""das"", ...",2023-01-26 21:33:37.347,2023-01-26 21:34:06.865999872,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,...,NaN,CLOZE_TEXT_INPUT,"{""id"": ""3FzjtkOlOh769ACiGbVzlf"", ""type"": ""CLOZ...",3283,3249,Inhalt von Nebensätzen,NaN,Content of subordinate clauses,NaN,0
866709,3761602,e233180b-07d4-4aec-9abb-fe4029e861de,415855,4YK0zbwWcdf76dxAb0FQeJ,177979,CORRECT,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""komme""...",2023-01-23 09:25:36.102,2023-01-23 09:26:07.860999936,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,...,NaN,CLOZE_TEXT_INPUT,"{""id"": ""4YK0zbwWcdf76dxAb0FQeJ"", ""type"": ""CLOZ...",3285,3249,Inhalt von Nebensätzen,NaN,Content of subordinate clauses,NaN,0


In [14]:
## selecting only the transactions related to the German questions
merged_de = merged_with_topics[merged_with_topics["math"]==0]
merged_de.to_csv(f"{BASE_DATA_DIR}/lernnavi/merged_deutsch.csv")

In [15]:
#writing a function to extract the difficulty of each question from the data
def get_difficulty(doc):
    res = None
    doc = json.loads(doc)
    if doc.get('metaData') != None:
        res = doc.get('metaData').get('estimatedDifficulty')
    return res
merged_de['difficulty'] = merged_de['content'].apply(lambda merged_de: get_difficulty(merged_de))

/tmp/ipykernel_13063/3609240770.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_de['difficulty'] = merged_de['content'].apply(lambda merged_de: get_difficulty(merged_de))


In [16]:
#assigning 2 scores to each question:

#the first is just related to the evaluation (0: incorrect, 0.5 : partial, 1:correct)
merged_de['score_stud'] = merged_de["evaluation"].apply(assign_score)

#the second is given by (first score)*(difficulty)
merged_de['score_stud2'] = merged_de['score_stud']*(merged_de['difficulty'])

merged_de

/tmp/ipykernel_13063/161134497.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_de['score_stud'] = merged_de["evaluation"].apply(assign_score)
/tmp/ipykernel_13063/161134497.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_de['score_stud2'] = merged_de['score_stud']*(merged_de['difficulty'])


,transaction_id,transaction_token,user_id,document_id,document_version,evaluation,input,start_time,commit_time,user_agent,...,topic_id_y,id,german_name,german_description,name,description,math,difficulty,score_stud,score_stud2
0,702616,d7f72afa-066c-49d3-877d-80e4bce9364f,391625,9wk5dtV2mF59odW0wCEYYc,75003,PARTIAL,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""Assona...",2021-06-02 15:48:45.228,2021-06-02 15:52:20.670000128,unknown,...,4256,2045,Rhetorik,NaN,Rhetoric,NaN,0,3.0,0.5,1.5
1,703810,b20883dc-dc03-4c25-8f0a-b625af28c041,388290,9wk5dtV2mF59odW0wCEYYc,75003,CORRECT,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""Metaph...",2021-06-04 05:50:50.265,2021-06-04 05:51:32.985999872,unknown,...,4256,2045,Rhetorik,NaN,Rhetoric,NaN,0,3.0,1.0,3.0
2,716303,490023d6-ceeb-4f0d-aa1f-2b207efac1fa,392845,9wk5dtV2mF59odW0wCEYYc,75003,WRONG,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""blumen...",2021-06-15 08:16:58.894,2021-06-15 08:17:43.192000000,unknown,...,4256,2045,Rhetorik,NaN,Rhetoric,NaN,0,3.0,0.0,0.0
3,727495,023b2fac-9e2b-4c54-9072-09065dddc56d,391776,9wk5dtV2mF59odW0wCEYYc,75003,WRONG,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": ["""", """",...",2021-07-01 08:28:54.139,2021-07-01 08:30:17.124000000,unknown,...,4256,2045,Rhetorik,NaN,Rhetoric,NaN,0,3.0,0.0,0.0
4,741050,20d9f392-c5ba-4756-bb24-0aad3c78a1b0,393730,6buw4x5E4Yt92KVfYANu0e,75016,CORRECT,"{""type"": ""OPEN_TASK"", ""openInput"": ""Trikolon\n""}",2021-07-30 14:42:42.929,2021-07-30 14:43:25.931000064,unknown,...,4256,2045,Rhetorik,NaN,Rhetoric,NaN,0,2.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866706,3473424,a643321c-1117-4275-b6fc-efac3ab3d040,413371,3FzjtkOlOh769ACiGbVzlf,166717,PARTIAL,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""das"", ...",2022-12-14 09:15:06.599,2022-12-14 09:15:50.023000064,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,...,3283,3249,Inhalt von Nebensätzen,NaN,Content of subordinate clauses,NaN,0,2.0,0.5,1.0
866707,3606166,1d4c1eb3-25fc-473d-9878-225ac31a0390,427529,3FzjtkOlOh769ACiGbVzlf,166717,PARTIAL,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""was"", ...",2023-01-05 22:11:49.222,2023-01-05 22:12:50.103000064,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,...,3283,3249,Inhalt von Nebensätzen,NaN,Content of subordinate clauses,NaN,0,2.0,0.5,1.0
866708,3812408,a56e13ed-e447-45f0-b80f-5f4044c6e238,404555,3FzjtkOlOh769ACiGbVzlf,166717,PARTIAL,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""das"", ...",2023-01-26 21:33:37.347,2023-01-26 21:34:06.865999872,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,...,3283,3249,Inhalt von Nebensätzen,NaN,Content of subordinate clauses,NaN,0,2.0,0.5,1.0
866709,3761602,e233180b-07d4-4aec-9abb-fe4029e861de,415855,4YK0zbwWcdf76dxAb0FQeJ,177979,CORRECT,"{""type"": ""CLOZE_TEXT"", ""clozeInputs"": [""komme""...",2023-01-23 09:25:36.102,2023-01-23 09:26:07.860999936,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,...,3285,3249,Inhalt von Nebensätzen,NaN,Content of subordinate clauses,NaN,0,2.0,1.0,2.0


In [27]:
tot_nw = merged_de[["user_id", "score_stud"]].groupby("user_id").sum()['score_stud']
tot_nw = tot_nw.to_frame()
tot_nw

,score_stud
user_id,
387604,149.0
387605,37.5
387615,5.0
387643,43.5
387644,59.0
...,...
431832,1.5
431836,11.5
431855,3.5


In [28]:
tot_w = merged_de[["user_id", "score_stud2"]].groupby("user_id").sum()['score_stud2']
tot_w = tot_w.to_frame()

In [29]:
users = users.merge(tot_nw, on="user_id")
users = users.merge(tot_w, on="user_id")
users

,user_id,counts,score_stud,score_stud2
0,387604,186,149.0,358.0
1,387605,53,37.5,69.0
2,387615,6,5.0,8.0
3,387643,53,43.5,109.5
4,387644,71,59.0,162.0
...,...,...,...,...
12841,431832,3,1.5,3.5
12842,431836,17,11.5,32.0
12843,431855,7,3.5,6.5
12844,431868,1,0.5,1.0


In [30]:
## computing the total number of questions, so to normalize the previously computed sums
counts = merged_de.groupby("user_id")["transaction_id"].count()
counts.to_frame()

,transaction_id
user_id,
387604,186
387605,53
387615,6
387643,53
387644,71
...,...
431832,3
431836,17
431855,7


In [33]:
users

,user_id,counts,score_stud,score_stud2,score_w,score_nw
0,387604,186,149.0,358.0,0.801075,1.924731
1,387605,53,37.5,69.0,0.707547,1.301887
2,387615,6,5.0,8.0,0.833333,1.333333
3,387643,53,43.5,109.5,0.820755,2.066038
4,387644,71,59.0,162.0,0.830986,2.281690
...,...,...,...,...,...,...
12841,431832,3,1.5,3.5,0.500000,1.166667
12842,431836,17,11.5,32.0,0.676471,1.882353
12843,431855,7,3.5,6.5,0.500000,0.928571
12844,431868,1,0.5,1.0,0.500000,1.000000


In [34]:
users = users.merge(counts, on="user_id")
users["counts"] = users["transaction_id"]
users = users.drop(columns=["transaction_id"])
users.reset_index()
users.set_index("user_id")
# users = users.drop(columns=['gender','canton','class_level', 'study', 'class_id'])
users

,user_id,counts,score_stud,score_stud2,score_w,score_nw
0,387604,186,149.0,358.0,0.801075,1.924731
1,387605,53,37.5,69.0,0.707547,1.301887
2,387615,6,5.0,8.0,0.833333,1.333333
3,387643,53,43.5,109.5,0.820755,2.066038
4,387644,71,59.0,162.0,0.830986,2.281690
...,...,...,...,...,...,...
12841,431832,3,1.5,3.5,0.500000,1.166667
12842,431836,17,11.5,32.0,0.676471,1.882353
12843,431855,7,3.5,6.5,0.500000,0.928571
12844,431868,1,0.5,1.0,0.500000,1.000000


In [35]:
users["score_w"] = users["score_stud"]/users["counts"]
users["score_nw"] = users["score_stud2"]/users["counts"]

In [36]:
users

,user_id,counts,score_stud,score_stud2,score_w,score_nw
0,387604,186,149.0,358.0,0.801075,1.924731
1,387605,53,37.5,69.0,0.707547,1.301887
2,387615,6,5.0,8.0,0.833333,1.333333
3,387643,53,43.5,109.5,0.820755,2.066038
4,387644,71,59.0,162.0,0.830986,2.281690
...,...,...,...,...,...,...
12841,431832,3,1.5,3.5,0.500000,1.166667
12842,431836,17,11.5,32.0,0.676471,1.882353
12843,431855,7,3.5,6.5,0.500000,0.928571
12844,431868,1,0.5,1.0,0.500000,1.000000


In [39]:
## merging the user dataframe with the average mastery scores for each student
mean_mastery.name = "mean_mastery"
u_final = users.merge(mean_mastery, on="user_id")
u_final

,user_id,counts,score_stud,score_stud2,score_w,score_nw,mean_mastery
0,387604,186,149.0,358.0,0.801075,1.924731,0.177370
1,387605,53,37.5,69.0,0.707547,1.301887,0.098331
2,387615,6,5.0,8.0,0.833333,1.333333,0.096108
3,387643,53,43.5,109.5,0.820755,2.066038,0.048141
4,387644,71,59.0,162.0,0.830986,2.281690,0.111754
...,...,...,...,...,...,...,...
12799,431832,3,1.5,3.5,0.500000,1.166667,0.000000
12800,431836,17,11.5,32.0,0.676471,1.882353,0.022056
12801,431855,7,3.5,6.5,0.500000,0.928571,0.000672
12802,431868,1,0.5,1.0,0.500000,1.000000,0.025663


In [40]:
u_final.to_csv(f"{BASE_DATA_DIR}/lernnavi/user_data.csv")

### Limitations and future directions (to be implemented in the final version of the project)

As previously stated, there are certain limitations associated with the features that have been extracted. Specifically, it should be noted that the mastery scores extracted are the most recent ones in the available student progress timeline. At the same time, the scores for each student were computed with all the available data. With this kind of preprocessing there's a certain (little) amount of "data leaking". However, this is fixed in ```feature_engineering_2.ipynb``` where we perform mastery-question timesteries matching. We also would like to test the following generalizations (in increasing order of difficulty):

1) the pair (known student, question not seen for that student)
2) the pair (unseen student, known question)
3) the pair (unseen student, unseen question)  

